In [24]:
#Importing the necessary packages (os, youtube_dl, editor from moviepy, sqlite3)
import os
import sys, youtube_dl
import moviepy.editor as mp
import sqlite3
from sqlite3 import Error

In [25]:
#Setting up the working directory
data_dir="C:\\Users\\devel\\OneDrive\\Documents\\Notes\\Python-Programming-1214\\Assignments"
os.chdir(data_dir)
os.getcwd()

'C:\\Users\\devel\\OneDrive\\Documents\\Notes\\Python-Programming-1214\\Assignments'

In [26]:
#Setting up the YouTube directory where the video file will be downloaded
YOUTUBE_DL_DIR = 'C:\\Users\\devel\\OneDrive\\Documents\\Notes\\Python-Programming-1214\\Assignments'

In [27]:
#.   Creating function download_video() to download the video from Youtube where we are specifying the link and specifying the name for the video that needs to be saved in the local
def download_video(videoId):
    ytbevideo = ['https://www.youtube.com/watch?v='+videoId]
    ytbe_opts = {'outtmpl': 'Project_Video'}

    with youtube_dl.YoutubeDL(ytbe_opts) as ydl:
        ydl.download(ytbevideo)

In [28]:
#Calling the function download_video() in the main program by passing the id of the video as parameter
if __name__ == "__main__":
    videoId = 'sVPYIRF9RCQ'
    download_video(videoId)

[youtube] sVPYIRF9RCQ: Downloading webpage


[download] Destination: Project_Video.f137
[download] 100% of 19.29MiB in 05:32                    
[download] Destination: Project_Video.f251
[download] 100% of 1.34MiB in 00:22                 
[ffmpeg] Merging formats into "Project_Video.mkv"
Deleting original file Project_Video.f137 (pass -k to keep)
Deleting original file Project_Video.f251 (pass -k to keep)


In [29]:
#Creating a subprocess using VideoFileClip for the above-saved video
clip = mp.VideoFileClip(r"C:\\Users\\devel\\OneDrive\\Documents\\Notes\\Python-Programming-1214\\Assignments\\Project_Video.mkv")

In [30]:
#Extracting the audio from the specified video and saving the audio clip as Project_audio.mp3 in the local path
clip.audio.write_audiofile(r"Project_audio.mp3")

MoviePy - Writing audio in Project_audio.mp3


MoviePy - Done.


In [31]:
cd C:\Users\devel\elite_blob_data

C:\Users\devel\elite_blob_data


In [32]:
os.getcwd()

'C:\\Users\\devel\\elite_blob_data'

In [33]:
#Connecting to the database and creating a table Elite_audio_uploads
try:
    
    # Connect to DB and create a cursor
    sqliteConnection = sqlite3.connect('app.db')
    cursor = sqliteConnection.cursor()
    print('DB Init')
    
    # Write a query and execute it with cursor
    query = 'CREATE TABLE IF NOT EXISTS Elite_audio_uploads (id integer PRIMARY KEY,file_name text NOT NULL,file_blob text NOT NULL);'
    cursor.execute(query)
  
    # Fetch and output result
    result = cursor.fetchall()
   
    # Close the cursor
    cursor.close()
  
# Handle errors
except sqlite3.Error as error:
    print('Error occured - ', error)
  
# Close DB Connection irrespective of success
# or failure
finally:
    
    if sqliteConnection:
        sqliteConnection.close()
        print('SQLite Connection closed')


DB Init
SQLite Connection closed


In [34]:
#Creating a function ‘insert_into_database()’ to insert the audio file details to the created table Elite_audio_uploads
def insert_into_database(file_path_name, file_blob): 
    try:
        conn = sqlite3.connect('app.db')
        print("[INFO] : Successful connection!")
        cur = conn.cursor()
        sql_insert_file_query = '''INSERT INTO Elite_audio_uploads(file_name, file_blob) VALUES(?, ?)'''
        cur = conn.cursor()
        cur.execute(sql_insert_file_query, (file_path_name, file_blob, ))
        conn.commit()
        print("[INFO] : The blob for ", file_path_name, " is in the database.") 
        last_updated_entry = cur.lastrowid
        return last_updated_entry
    # Handle errors
    except Error as e:
        print(e)
    # Close DB Connection irrespective of success
    finally:
        if conn:
            conn.close()
        else:
            error = "Something is wrong here."


In [35]:
#Creating a function ‘convert_into_binary()’ to convert the targeted audio file into binary format
def convert_into_binary(file_path):
    with open(file_path, 'rb') as file:
        binary = file.read()
    return binary    

In [36]:
#Creating a function ‘read_blob_data()’ to read the saved audio record from the database
def read_blob_data(entry_id):
    try:
        conn = sqlite3.connect('app.db')
        cur = conn.cursor()
        print("[INFO] : Connected to SQLite to read_blob_data")
        sql_fetch_blob_query = """SELECT * from Elite_audio_uploads where id = ?"""
        cur.execute(sql_fetch_blob_query, (entry_id,))
        record = cur.fetchall()
        #Extracting the records.
        for row in record:
            converted_file_name = row[1]
            audio_binarycode  = row[2]
                        
            last_slash_index = converted_file_name.rfind("/") + 1 
            final_file_name = converted_file_name[last_slash_index:] 
            write_to_file(audio_binarycode, final_file_name)
            print("Audio successfully stored in the local folder. \n")
        cur.close()
    # Handle errors
    except sqlite3.Error as error:
        print("Failed to read audio blob data from sqlite table", error)
    # Close DB Connection irrespective of success
    finally:
         if conn:
            conn.close()

In [39]:
#Creating a function ‘write_to_file()’ to write back the audio file read from the database to the local folder
def write_to_file(binary_data, file_name):
    outFileName="C:\\Users\\devel\\OneDrive\\Documents\\Notes\\Python-Programming-1214\\Assignments\\Output_Audio_from_database\\Project_Audio.mp3"
    with open(outFileName, 'wb') as file:
        file.write(binary_data)
    print("The audio file has been written to the project directory: ", file_name)

In [40]:
#Creating a function ‘main()’ where the path of the file is being accepted from the user and the functions convert_into_binary, insert_into_database and read_blob_data are called
def main():
    print("Python Project!!")
    file_path_name = input("Enter full audio file path:\n") 
    file_blob = convert_into_binary(file_path_name)
    
    last_updated_entry = insert_into_database(file_path_name, file_blob)
    read_blob_data(last_updated_entry)

#Calling the above-created function from the main program
if __name__ == "__main__":
    main()

Python Project!!
Enter full audio file path:
C:\Users\devel\OneDrive\Documents\Notes\Python-Programming-1214\Assignments\Project_Audio.mp3
[INFO] : Successful connection!
[INFO] : The blob for  C:\Users\devel\OneDrive\Documents\Notes\Python-Programming-1214\Assignments\Project_Audio.mp3  is in the database.
[INFO] : Connected to SQLite to read_blob_data
The audio file has been written to the project directory:  C:\Users\devel\OneDrive\Documents\Notes\Python-Programming-1214\Assignments\Project_Audio.mp3
Audio successfully stored in the local folder. 

